In [5]:
import pylab as pl
import pandas as pd
import numpy as np
import os
import json

%pylab inline

if os.getenv ('PUI2018') is None:
    print ("Must set env variable PUI2018")
if os.getenv ('PUIDATA') is None:
    print ("Must set env variable PUI2018")


def getCitiBikeCSV(datestring):
    print ("Downloading", datestring)
    ### First I will heck that it is not already there
    if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.csv"):
        if os.path.isfile(datestring + "-citibike-tripdata.csv"):
            # if in the current dir just move it
            if os.system("mv " + datestring + "-citibike-tripdata.csv " + os.getenv("PUIDATA")):
                print ("Error moving file!, Please check!")
        #otherwise start looking for the zip file
        else:
            if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.zip"):
                if not os.path.isfile(datestring + "-citibike-tripdata.zip"):
                    os.system("curl -O https://s3.amazonaws.com/tripdata/" + datestring + "-citibike-tripdata.zip")
                ###  To move it I use the os.system() functions to run bash commands with arguments
                os.system("mv " + datestring + "-citibike-tripdata.zip " + os.getenv("PUIDATA"))
            ### unzip the csv 
            os.system("unzip " + os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.zip")
            ## NOTE: old csv citibike data had a different name structure. 
            if '2014' in datestring:
                os.system("mv " + datestring[:4] + '-' +  datestring[4:] + 
                          "\ -\ Citi\ Bike\ trip\ data.csv " + datestring + "-citibike-tripdata.csv")
            os.system("mv " + datestring + "-citibike-tripdata.csv " + os.getenv("PUIDATA"))
    ### One final check:
    if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.csv"):
        print ("WARNING!!! something is wrong: the file is not there!")

    else:
        print ("file in place, you can continue")

Populating the interactive namespace from numpy and matplotlib
Must set env variable PUI2018


In [6]:
datestring = '201501'
getCitiBikeCSV(datestring)

file in place, you can continue


In [7]:
df = pd.read_csv(os.getenv("PUIDATA") + "/" + datestring + '-citibike-tripdata.csv')
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1346,1/1/2015 0:01,1/1/2015 0:24,455,1 Ave & E 44 St,40.750020,-73.969053,265,Stanton St & Chrystie St,40.722293,-73.991475,18660,Subscriber,1960.0,2
1,363,1/1/2015 0:02,1/1/2015 0:08,434,9 Ave & W 18 St,40.743174,-74.003664,482,W 15 St & 7 Ave,40.739355,-73.999318,16085,Subscriber,1963.0,1
2,346,1/1/2015 0:04,1/1/2015 0:10,491,E 24 St & Park Ave S,40.740964,-73.986022,505,6 Ave & W 33 St,40.749013,-73.988484,20845,Subscriber,1974.0,1
3,182,1/1/2015 0:04,1/1/2015 0:07,384,Fulton St & Waverly Ave,40.683178,-73.965964,399,Lafayette Ave & St James Pl,40.688515,-73.964763,19610,Subscriber,1969.0,1
4,969,1/1/2015 0:05,1/1/2015 0:21,474,5 Ave & E 29 St,40.745168,-73.986831,432,E 7 St & Avenue A,40.726218,-73.983799,20197,Subscriber,1977.0,1


# H0: There is no correlation between the trip duration and the time of the trip whether it's day or night
# H1: There is a significant correlation between the trip duration and the time of the trip whether it's a day or night
# alpha = 0.05

In [8]:
df['date'] = pd.to_datetime(df['starttime'])
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,date
0,1346,1/1/2015 0:01,1/1/2015 0:24,455,1 Ave & E 44 St,40.750020,-73.969053,265,Stanton St & Chrystie St,40.722293,-73.991475,18660,Subscriber,1960.0,2,2015-01-01 00:01:00
1,363,1/1/2015 0:02,1/1/2015 0:08,434,9 Ave & W 18 St,40.743174,-74.003664,482,W 15 St & 7 Ave,40.739355,-73.999318,16085,Subscriber,1963.0,1,2015-01-01 00:02:00
2,346,1/1/2015 0:04,1/1/2015 0:10,491,E 24 St & Park Ave S,40.740964,-73.986022,505,6 Ave & W 33 St,40.749013,-73.988484,20845,Subscriber,1974.0,1,2015-01-01 00:04:00
3,182,1/1/2015 0:04,1/1/2015 0:07,384,Fulton St & Waverly Ave,40.683178,-73.965964,399,Lafayette Ave & St James Pl,40.688515,-73.964763,19610,Subscriber,1969.0,1,2015-01-01 00:04:00
4,969,1/1/2015 0:05,1/1/2015 0:21,474,5 Ave & E 29 St,40.745168,-73.986831,432,E 7 St & Avenue A,40.726218,-73.983799,20197,Subscriber,1977.0,1,2015-01-01 00:05:00


In [9]:
df.columns

Index(['tripduration', 'starttime', 'stoptime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype',
       'birth year', 'gender', 'date'],
      dtype='object')

In [10]:
df.drop(['starttime', 'stoptime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype',
       'birth year', 'gender'], axis=1, inplace=True)
df.head()

,tripduration,date
0,1346,2015-01-01 00:01:00
1,363,2015-01-01 00:02:00
2,346,2015-01-01 00:04:00
3,182,2015-01-01 00:04:00
4,969,2015-01-01 00:05:00


In [11]:
df['tripduration']= np.divide(df['tripduration'], 60)

In [12]:
df.describe()

,tripduration
count,285552.000000
mean,10.905426
std,15.012932
min,1.000000
25%,5.566667
50%,8.400000
75%,12.866667
max,717.050000


In [14]:
#df["dayornight"] = 7 < df.date[].hour < 19
df["hour"] = df["date"].dt.hour
df

,tripduration,date,hour
0,22.433333,2015-01-01 00:01:00,0
1,6.050000,2015-01-01 00:02:00,0
2,5.766667,2015-01-01 00:04:00,0
3,3.033333,2015-01-01 00:04:00,0
4,16.150000,2015-01-01 00:05:00,0
5,8.266667,2015-01-01 00:07:00,0
6,2.533333,2015-01-01 00:07:00,0
7,19.716667,2015-01-01 00:08:00,0
8,14.100000,2015-01-01 00:09:00,0
9,9.600000,2015-01-01 00:10:00,0


In [15]:
#Day trips are between 7:00AM and 7:00PM
day = df[df["hour"] >=7]
day = day[day["hour"] <19]
day.describe()

,tripduration,hour
count,222548.000000,222548.000000
mean,11.086115,12.938256
std,13.925147,3.689274
min,1.000000,7.000000
25%,5.666667,9.000000
50%,8.550000,13.000000
75%,13.116667,16.000000
max,717.050000,18.000000


In [16]:
#Night trips are between 7:00PM and 7:00AM
night1 = df[df["hour"] <7]
#night = day[day["hour"] <19]
night2 = df[df["hour"] >=19]
night = night1.append(night2)
night.describe()

,tripduration,hour
count,63004.000000,63004.000000
mean,10.267183,16.062996
std,18.331965,7.308226
min,1.000000,0.000000
25%,5.250000,6.000000
50%,7.883333,19.000000
75%,11.950000,21.000000
max,714.633333,23.000000


In [17]:
import scipy.stats

In [34]:
#Pearson's Correlation Test
sampleday = day.sample(n=len(night), random_state=999)
pearson = scipy.stats.pearsonr(sampleday.tripduration,night.tripduration)
print(pearson)

if int(pearson[0]) == 0:
    print("No Correlation found! Reject the NULL hypothesis")
else:
     print("Correlation is found! Accept the NULL hypothesis")   

(0.001157192271907443, 0.77146701722424293)
No Correlation found! Reject the NULL hypothesis


In [35]:
#Spearman's Correlation Test
sampleday = day.sample(n=len(night), random_state=999)
spearman = scipy.stats.spearmanr(sampleday.tripduration,night.tripduration)
print(spearman)

if int(spearman[0]) == 0:
    print("No Correlation found! Reject the NULL hypothesis")
else:
     print("Correlation is found! Accept the NULL hypothesis")   

SpearmanrResult(correlation=0.0020950791817358973, pvalue=0.59898024466773903)
No Correlation found! Reject the NULL hypothesis


In [36]:
#K-S Correlation Test
sampleday = day.sample(n=len(night), random_state=999)
k_s = scipy.stats.ks_2samp(sampleday.tripduration,night.tripduration)
print(k_s)

if int(k_s[1]) == 0:
    print("No Correlation found! Reject the NULL hypothesis")
else:
     print("Correlation is found! Accept the NULL hypothesis") 

Ks_2sampResult(statistic=0.054663195987556312, pvalue=2.6888407766015938e-82)
No Correlation found! Reject the NULL hypothesis
